# Module 21: Time-to-Collision Fundamentals (No-Code Lab)

### ⚠ If you run into issues, go to Kernel -> Restart and Clear Output

TTC estimates how much time remains before impact if current speed and distance continue.

**Formula:** `TTC = distance / speed`


## Section 1: Startup (Prebuilt)

In [ ]:

import sys, os, time, cv2
from IPython.display import display, Image, HTML, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI, VESCStudentAPI

ai_api = AIStudentAPI()
if not ai_api.start_camera():
    raise RuntimeError('Camera failed to start. Re-run this cell.')
if not ai_api.start_buzzer():
    raise RuntimeError('Failed to initialize buzzer on GPIO17')

vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    deadline = time.time() + 10.0
    controllers = []
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)
    if controllers:
        vesc = vesc_api.get_controller(controllers[0])

print('✅ TTC lab ready')


## Section 2: Adjustable Parameters

In [ ]:

TARGET_LABEL = 'person'      #object you want to detect
MIN_CONFIDENCE = 0.70        #AI certainty of the object 

# Distance model: estimated_distance_m = KNOWN_HEIGHT_M * FOCAL_PX / box_height_px
KNOWN_HEIGHT_M = 1.70        #height
FOCAL_PX = 700.0             #focal px of camera

WHEEL_RADIUS_M = 0.04        #wheel radius in m
GEAR_RATIO = 1.0             #gear ratio of pulleys, needs changed

TTC_WARNING_S = 3.0          #time in s of TTC warning threshold
TTC_CRITICAL_S = 1.2         #time in s of TTC critical threshold
RUN_DELAY_S = 0.10           #run delay


## Section 3: Live TTC Monitor

In [ ]:

    def rpm_to_speed_mps(rpm):
        wheel_circ = 2.0 * 3.14159 * WHEEL_RADIUS_M
        return abs(rpm) * wheel_circ / 60.0 / max(GEAR_RATIO, 0.01)

    def estimate_distance_m(box_h):
        box_h = max(float(box_h), 1.0)
        return (KNOWN_HEIGHT_M * FOCAL_PX) / box_h

    last_beep = 0.0

    try:
        print('Running TTC monitor... press Interrupt (■) to stop.')
        while True:
            frame, detections = ai_api.get_frame_and_detections()
            rpm = vesc.get_rpm() if vesc else 0.0
            speed_mps = rpm_to_speed_mps(rpm)

            best_dist = float('inf')
            for det in detections:
                x, y, w, h = det['box']
                label = det['label']
                conf = float(det['confidence'])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (4, 23, 115), 2)
                cv2.putText(frame, f"{label}:{conf:.2f}", (x, max(15, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (4, 23, 115), 2)
                if label == TARGET_LABEL and conf >= MIN_CONFIDENCE:
                    best_dist = min(best_dist, estimate_distance_m(h))

            ttc = best_dist / speed_mps if speed_mps > 0.05 and best_dist < float('inf') else float('inf')

            status = 'SAFE'
            if ttc <= TTC_CRITICAL_S:
                status = 'CRITICAL'
            elif ttc <= TTC_WARNING_S:
                status = 'WARNING'

            now = time.time()
            if status in ('WARNING', 'CRITICAL') and now - last_beep > 2.0:
                ai_api.buzzer_beep(0.10, 0.08, 1 if status == 'WARNING' else 3)
                last_beep = now

            panel = (
                f"Status={status} | RPM={rpm:.0f} | Speed={speed_mps:.2f} m/s | "
                f"Distance={best_dist if best_dist < float('inf') else -1:.2f} m | "
                f"TTC={ttc if ttc < float('inf') else -1:.2f} s"
            )
            cv2.putText(frame, panel, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.52, (255, 255, 255), 2)

            _, img_encoded = cv2.imencode('.jpeg', frame)
            clear_output(wait=True)
            display(Image(data=img_encoded.tobytes()))
            print(panel)
            time.sleep(RUN_DELAY_S)

    except KeyboardInterrupt:
        print('
Stopped.')



## Section 4: ADAS Knowledge Check (No Coding)

1. Why does TTC drop quickly as speed increases?
2. Why can TTC be high-risk even when an object is still far away?
3. Why should warning and critical TTC thresholds be different?


In [ ]:

if 'ai_api' in locals():
    ai_api.shutdown_hardware(stop_camera=True, stop_buzzer=True)
    print('Camera + buzzer released.')
if 'vesc_api' in locals() and vesc_api.is_running():
    vesc_api.stop()
    print('VESC API closed.')
